In [1]:
import wandb
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda

import gym
import argparse
import numpy as np

tf.keras.backend.set_floatx('float64')

# parser = argparse.ArgumentParser()
# parser.add_argument('--gamma', type=float, default=0.99)
# parser.add_argument('--update_interval', type=int, default=5)
# parser.add_argument('--actor_lr', type=float, default=0.0005)
# parser.add_argument('--critic_lr', type=float, default=0.001)
# parser.add_argument('--clip_ratio', type=float, default=0.1)
# parser.add_argument('--lmbda', type=float, default=0.95)
# parser.add_argument('--epochs', type=int, default=3)

# args = parser.parse_args()

# class Args:
#     gamma = 0.99
#     update_interval = 5
#     actor_lr = 0.0005
#     critic_lr = 0.001
#     batch_size = 64
#     clip_ratio = 0.1
#     lmbda = 0.95
#     intervals = 3
    
#     episodes = 10
#     N = 3
#     epochs = 100

# args = Args()


In [2]:
class Actor:
    def __init__(self, state_dim, action_dim, action_bound, std_bound):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.action_bound = action_bound
        self.std_bound = std_bound
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(wandb.config.actor_lr)

    def get_action(self, state):
        state = np.reshape(state, [1, self.state_dim])
        mu, std = self.model.predict(state)
        action = np.random.normal(mu[0], std[0], size=self.action_dim)
        action = np.clip(action, -self.action_bound, self.action_bound)
        log_policy = self.log_pdf(mu, std, action)

        return log_policy, action

    def log_pdf(self, mu, std, action):
        std = tf.clip_by_value(std, self.std_bound[0], self.std_bound[1])
        var = std ** 2
        log_policy_pdf = -0.5 * (action - mu) ** 2 / \
            var - 0.5 * tf.math.log(var * 2 * np.pi)
        return tf.reduce_sum(log_policy_pdf, 1, keepdims=True)

    def create_model(self):
        state_input = Input((self.state_dim,))
        dense_1 = Dense(wandb.config.actor['layer1'], activation='relu')(state_input)
        dense_2 = Dense(wandb.config.actor['layer2'], activation='relu')(dense_1)
        out_mu = Dense(self.action_dim, activation='tanh')(dense_2)
        mu_output = Lambda(lambda x: x * self.action_bound)(out_mu)
        std_output = Dense(self.action_dim, activation='softplus')(dense_2)
        return tf.keras.models.Model(state_input, [mu_output, std_output])

    def compute_loss(self, log_old_policy, log_new_policy, actions, gaes):
        ratio = tf.exp(log_new_policy - tf.stop_gradient(log_old_policy))
        gaes = tf.stop_gradient(gaes)
        clipped_ratio = tf.clip_by_value(
            ratio, 1.0-wandb.config.clip_ratio, 1.0+wandb.config.clip_ratio)
        surrogate = -tf.minimum(ratio * gaes, clipped_ratio * gaes)
        return tf.reduce_mean(surrogate)

    def train(self, log_old_policy, states, actions, gaes):
        with tf.GradientTape() as tape:
            mu, std = self.model(states, training=True)
            log_new_policy = self.log_pdf(mu, std, actions)
            loss = self.compute_loss(
                log_old_policy, log_new_policy, actions, gaes)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss


In [3]:
class Critic:
    def __init__(self, state_dim):
        self.state_dim = state_dim
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(wandb.config.critic_lr)

    def create_model(self):
        return tf.keras.Sequential([
            Input((self.state_dim,)),
            Dense(wandb.config.critic['layer1'], activation='relu'),
            Dense(wandb.config.critic['layer2'], activation='relu'),
            Dense(wandb.config.critic['layer3'], activation='relu'),
            Dense(1, activation='linear')
        ])

    def compute_loss(self, v_pred, td_targets):
        mse = tf.keras.losses.MeanSquaredError()
        return mse(td_targets, v_pred)

    def train(self, states, td_targets):
        with tf.GradientTape() as tape:
            v_pred = self.model(states, training=True)
            assert v_pred.shape == td_targets.shape
            loss = self.compute_loss(v_pred, tf.stop_gradient(td_targets))
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss


In [4]:
class Agent:
    def __init__(self, env, iden = 0):
        self.env = env
        self.state_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.shape[0]
        self.action_bound = self.env.action_space.high[0]
        self.std_bound = [1e-2, 1.0]

        self.actor_opt = tf.keras.optimizers.Adam(wandb.config.actor_lr)
        self.critic_opt = tf.keras.optimizers.Adam(wandb.config.critic_lr)
        self.actor = Actor(self.state_dim, self.action_dim,
                           self.action_bound, self.std_bound)
        self.critic = Critic(self.state_dim)
        
        self.iden = iden

    def gae_target(self, rewards, v_values, next_v_value, done):
        n_step_targets = np.zeros_like(rewards)
        gae = np.zeros_like(rewards)
        gae_cumulative = 0
        forward_val = 0

        if not done:
            forward_val = next_v_value

        for k in reversed(range(0, len(rewards))):
            delta = rewards[k] + wandb.config.gamma * forward_val - v_values[k]
            gae_cumulative = wandb.config.gamma * wandb.config.lmbda * gae_cumulative + delta
            gae[k] = gae_cumulative
            forward_val = v_values[k]
            n_step_targets[k] = gae[k] + v_values[k]
        return gae, n_step_targets

    def list_to_batch(self, list):
        batch = list[0]
        for elem in list[1:]:
            batch = np.append(batch, elem, axis=0)
        return batch

    def train(self, max_episodes=1000):
        for ep in range(max_episodes):
            state_batch = []
            action_batch = []
            reward_batch = []
            old_policy_batch = []

            episode_reward, done = 0, False

            state = self.env.reset()

            while not done:
                # self.env.render()
                log_old_policy, action = self.actor.get_action(state)

                next_state, reward, done, _ = self.env.step(action)

                state = np.reshape(state, [1, self.state_dim])
                action = np.reshape(action, [1, 1])
                next_state = np.reshape(next_state, [1, self.state_dim])
                reward = np.reshape(reward, [1, 1])
                log_old_policy = np.reshape(log_old_policy, [1, 1])

                state_batch.append(state)
                action_batch.append(action)
                reward_batch.append((reward+8)/8)
                old_policy_batch.append(log_old_policy)

                if len(state_batch) >= wandb.config.update_interval or done:
                    states = self.list_to_batch(state_batch)
                    actions = self.list_to_batch(action_batch)
                    rewards = self.list_to_batch(reward_batch)
                    old_policys = self.list_to_batch(old_policy_batch)

                    v_values = self.critic.model.predict(states)
                    next_v_value = self.critic.model.predict(next_state)

                    gaes, td_targets = self.gae_target(
                        rewards, v_values, next_v_value, done)

                    for epoch in range(wandb.config.intervals):
                        actor_loss = self.actor.train(
                            old_policys, states, actions, gaes)
                        critic_loss = self.critic.train(states, td_targets)

                    state_batch = []
                    action_batch = []
                    reward_batch = []
                    old_policy_batch = []

                episode_reward += reward[0][0]
                state = next_state[0]

            print('EP{} EpisodeReward={}'.format(ep, episode_reward))
            wandb.log({'Reward' + str(self.iden): episode_reward})
        
        return episode_reward


In [7]:
if __name__ == "__main__":
    
    try: wandb.finish()
    except: pass
    
    ####configurations
    wandb.init(name='PPO-single-long', project="deep-rl-tf2")
    env_name = 'Pendulum-v0'

    
    wandb.config.gamma = 0.99
    wandb.config.update_interval = 5
    wandb.config.actor_lr = 0.0005
    wandb.config.critic_lr = 0.001
    wandb.config.batch_size = 64
    wandb.config.clip_ratio = 0.1
    wandb.config.lmbda = 0.95
    wandb.config.intervals = 3
    
    wandb.config.episodes = 5
    wandb.config.num = 1
    wandb.config.epochs = 200

    wandb.config.actor = {'layer1': 32, 'layer2' : 32}
    wandb.config.critic = {'layer1': 32, 'layer2' : 32, 'layer3': 16}
    
    print(wandb.config)
    
    # main run    
    N = wandb.config.num
    agents = []
    
    # set up the agent
    for i in range(N):
        env_t = gym.make(env_name)
        agents.append(Agent(env_t, i))

    # start the training
    for z in range(wandb.config.epochs):

        reward = 0
        # train the agent
        for j in range(len(agents)):
            print('Training Agent {}'.format(agents[j].iden))
            reward += agents[j].train(wandb.config.episodes)
    
        reward = reward / N
        print('Epoch={}\t Average reward={}'.format(z, reward))
        wandb.log({'batch': z, 'Epoch': reward})


        # get the average - actor and critic
        critic_avg = []
        actor_avg = []

        for i in range(len(agents[0].actor.model.get_weights())):
            
            actor_t = agents[0].actor.model.get_weights()[i]

            for j in range(1, N):
                actor_t += agents[j].actor.model.get_weights()[i]

            actor_t = actor_t / N
            actor_avg.append(actor_t)


        for i in range(len(agents[0].critic.model.get_weights())):
            critic_t = agents[0].critic.model.get_weights()[i]

            for j in range(1, N):
                critic_t += agents[j].critic.model.get_weights()[i]

            critic_t = critic_t / N
            critic_avg.append(critic_t)


        # set the average
        for j in range(N):
            agents[j].actor.model.set_weights(actor_avg)
            agents[j].critic.model.set_weights(critic_avg)


    # wrtie things out
    for j in range(N):
        with open("agent{}-actor.txt".format(j), "w") as f:
            f.write(str(agents[j].actor.model.get_weights()))
            f.close()
        wandb.save("agent{}-actor.txt".format(j))
        
        
        with open("agent{}-critic.txt".format(j), "w") as f:
            f.write(str(agents[j].critic.model.get_weights()))
            f.close()
        wandb.save("agent{}-critic.txt".format(j))

    
    wandb.finish()

{'gamma': 0.99, 'update_interval': 5, 'actor_lr': 0.0005, 'critic_lr': 0.001, 'batch_size': 64, 'clip_ratio': 0.1, 'lmbda': 0.95, 'intervals': 3, 'episodes': 5, 'num': 1, 'epochs': 200, 'actor': {'layer1': 32, 'layer2': 32}, 'critic': {'layer1': 32, 'layer2': 32, 'layer3': 16}}
Training Agent 0
EP0 EpisodeReward=-1604.323542328736
EP1 EpisodeReward=-1764.8738976768589
EP2 EpisodeReward=-1453.0758846940448
EP3 EpisodeReward=-1567.0979561135896
EP4 EpisodeReward=-1574.3512868836403
Epoch=0	 Average reward=-1574.3512868836403
Training Agent 0
EP0 EpisodeReward=-1351.4573219404592
EP1 EpisodeReward=-1166.0763621272197
EP2 EpisodeReward=-1507.028712534297
EP3 EpisodeReward=-1167.1190951940046
EP4 EpisodeReward=-1591.3551537959083
Epoch=1	 Average reward=-1591.3551537959083
Training Agent 0
EP0 EpisodeReward=-1447.0965414388672
EP1 EpisodeReward=-1504.6176026852056
EP2 EpisodeReward=-1204.0246770107337
EP3 EpisodeReward=-1522.700035715799
EP4 EpisodeReward=-1313.6323422939813
Epoch=2	 Averag

EP0 EpisodeReward=-1182.9311168156446
EP1 EpisodeReward=-1454.1287607421423
EP2 EpisodeReward=-1636.8647431486334
EP3 EpisodeReward=-1053.713762417968
EP4 EpisodeReward=-1640.231465543899
Epoch=32	 Average reward=-1640.231465543899
Training Agent 0
EP0 EpisodeReward=-1505.3062844307055
EP1 EpisodeReward=-1035.144828920091
EP2 EpisodeReward=-1620.1072749974508
EP3 EpisodeReward=-1655.1229541260973
EP4 EpisodeReward=-1336.234417813011
Epoch=33	 Average reward=-1336.234417813011
Training Agent 0
EP0 EpisodeReward=-1214.7538703747578
EP1 EpisodeReward=-1247.2556329420358
EP2 EpisodeReward=-1050.1352479481095
EP3 EpisodeReward=-1487.8134269457678
EP4 EpisodeReward=-1483.9670030914924
Epoch=34	 Average reward=-1483.9670030914924
Training Agent 0
EP0 EpisodeReward=-1414.1248447521114
EP1 EpisodeReward=-1175.284304530833
EP2 EpisodeReward=-961.8884125569509
EP3 EpisodeReward=-1573.9500418838877
EP4 EpisodeReward=-1586.2460182048467
Epoch=35	 Average reward=-1586.2460182048467
Training Agent 0


EP0 EpisodeReward=-1644.8578350669875
EP1 EpisodeReward=-1495.3176148492623
EP2 EpisodeReward=-1494.8635967847604
EP3 EpisodeReward=-1505.692886933457
EP4 EpisodeReward=-1626.9483827933284
Epoch=65	 Average reward=-1626.9483827933284
Training Agent 0
EP0 EpisodeReward=-1158.5290668561324
EP1 EpisodeReward=-1493.7238997303261
EP2 EpisodeReward=-1641.7696212780793
EP3 EpisodeReward=-1627.3284955485485
EP4 EpisodeReward=-1585.4976105531568
Epoch=66	 Average reward=-1585.4976105531568
Training Agent 0
EP0 EpisodeReward=-1493.5507085466486
EP1 EpisodeReward=-1515.918555809698
EP2 EpisodeReward=-1654.4947006519476
EP3 EpisodeReward=-1497.1618781698237
EP4 EpisodeReward=-972.5273339183657
Epoch=67	 Average reward=-972.5273339183657
Training Agent 0
EP0 EpisodeReward=-1567.362940635568
EP1 EpisodeReward=-1459.149750821978
EP2 EpisodeReward=-1621.4048984939466
EP3 EpisodeReward=-1354.1468194266022
EP4 EpisodeReward=-1644.1728254131801
Epoch=68	 Average reward=-1644.1728254131801
Training Agent 

EP0 EpisodeReward=-945.6973279776801
EP1 EpisodeReward=-1419.4920076657784
EP2 EpisodeReward=-1373.6615417569349
EP3 EpisodeReward=-1496.593649081974
EP4 EpisodeReward=-988.8738288516751
Epoch=98	 Average reward=-988.8738288516751
Training Agent 0
EP0 EpisodeReward=-1556.7815596327075
EP1 EpisodeReward=-1492.9129868157092
EP2 EpisodeReward=-1175.522051807821
EP3 EpisodeReward=-960.3938420533602
EP4 EpisodeReward=-1267.515658960753
Epoch=99	 Average reward=-1267.515658960753
Training Agent 0
EP0 EpisodeReward=-959.2866756063029
EP1 EpisodeReward=-1645.5506873027064
EP2 EpisodeReward=-1201.7088477433113
EP3 EpisodeReward=-1592.1164819513658
EP4 EpisodeReward=-1651.7524956041398
Epoch=100	 Average reward=-1651.7524956041398
Training Agent 0
EP0 EpisodeReward=-1079.9189621238834
EP1 EpisodeReward=-1497.9184965009988
EP2 EpisodeReward=-1251.1047614224342
EP3 EpisodeReward=-1646.9602126976104
EP4 EpisodeReward=-1649.4307191046694
Epoch=101	 Average reward=-1649.4307191046694
Training Agent 0

EP0 EpisodeReward=-1650.1811031311315
EP1 EpisodeReward=-1635.272195990819
EP2 EpisodeReward=-823.5640747342078
EP3 EpisodeReward=-1619.8388053387716
EP4 EpisodeReward=-831.3645909131121
Epoch=131	 Average reward=-831.3645909131121
Training Agent 0
EP0 EpisodeReward=-1279.188171131542
EP1 EpisodeReward=-1615.616249684792
EP2 EpisodeReward=-1409.9552413474926
EP3 EpisodeReward=-1627.4101275882044
EP4 EpisodeReward=-1339.3613990148065
Epoch=132	 Average reward=-1339.3613990148065
Training Agent 0
EP0 EpisodeReward=-1504.026678353535
EP1 EpisodeReward=-1391.1695146618918
EP2 EpisodeReward=-1393.6832066482482
EP3 EpisodeReward=-1633.1625469205767
EP4 EpisodeReward=-1657.246842615955
Epoch=133	 Average reward=-1657.246842615955
Training Agent 0
EP0 EpisodeReward=-1240.4820676077288
EP1 EpisodeReward=-1655.1667547097052
EP2 EpisodeReward=-1334.317767430322
EP3 EpisodeReward=-1519.503088778884
EP4 EpisodeReward=-1498.5391944949565
Epoch=134	 Average reward=-1498.5391944949565
Training Agent 0

EP0 EpisodeReward=-1265.8384804667905
EP1 EpisodeReward=-1459.0097886871572
EP2 EpisodeReward=-1551.0783660659667
EP3 EpisodeReward=-1490.9796059815644
EP4 EpisodeReward=-1197.774197836602
Epoch=164	 Average reward=-1197.774197836602
Training Agent 0
EP0 EpisodeReward=-1571.7377870430457
EP1 EpisodeReward=-1502.1113248148463
EP2 EpisodeReward=-1124.580260689107
EP3 EpisodeReward=-1497.327454622587
EP4 EpisodeReward=-1649.080181753957
Epoch=165	 Average reward=-1649.080181753957
Training Agent 0
EP0 EpisodeReward=-1167.3766467664116
EP1 EpisodeReward=-1639.7962567591846
EP2 EpisodeReward=-1019.2268912164259
EP3 EpisodeReward=-1165.621334663276
EP4 EpisodeReward=-1649.0390207041867
Epoch=166	 Average reward=-1649.0390207041867
Training Agent 0
EP0 EpisodeReward=-1657.5383374776698
EP1 EpisodeReward=-1442.1279597661583
EP2 EpisodeReward=-1424.969982213314
EP3 EpisodeReward=-1535.9870813343516
EP4 EpisodeReward=-948.3177185259486
Epoch=167	 Average reward=-948.3177185259486
Training Agent 

EP4 EpisodeReward=-1175.3820461139994
Epoch=196	 Average reward=-1175.3820461139994
Training Agent 0
EP0 EpisodeReward=-1499.624834190526
EP1 EpisodeReward=-1512.4955576362802
EP2 EpisodeReward=-1290.3207060086938
EP3 EpisodeReward=-1650.8950370666755
EP4 EpisodeReward=-1488.2100991371801
Epoch=197	 Average reward=-1488.2100991371801
Training Agent 0
EP0 EpisodeReward=-947.7084674232075
EP1 EpisodeReward=-1518.1609479675137
EP2 EpisodeReward=-1377.0016566092963
EP3 EpisodeReward=-1494.5737143921624
EP4 EpisodeReward=-1654.8268486545685
Epoch=198	 Average reward=-1654.8268486545685
Training Agent 0
EP0 EpisodeReward=-1656.7361612959462
EP1 EpisodeReward=-1595.5762209019827
EP2 EpisodeReward=-1343.9552044854227
EP3 EpisodeReward=-1481.1374377903874
EP4 EpisodeReward=-1488.6522386863169
Epoch=199	 Average reward=-1488.6522386863169


Reward0,-1488.65224
_step,1199
_runtime,7864
_timestamp,1605836301
batch,199
Epoch,-1488.65224


Reward0,▂▃▃▃▆▂▁▆▃▄▂▁▁▃▁█▅▄▁▇█▆▄▄▃▆▁█▅▄▇▃▁█▃▄▂▁▁▂
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
batch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Epoch,▂▃▃▃▄▁▁▂▁▁▁▁▇█▁▆▂▄▄▂▁▁▄▆▂▆▁▁▅▁▁▄▂█▄▁▂█▃▃
